In [6]:

# !pip install webdriver_manager

     ---------------------------------------- 62.8/62.8 kB 3.3 MB/s eta 0:00:00
  Using cached python_dotenv-0.20.0-py3-none-any.whl (17 kB)


In [76]:
# from scraping import selenium_run, auto_login
import json
import math
from time import sleep

import pandas as pd
import pyautogui as pg
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


def selenium_run():
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    url = 'https://code4biz.slack.com/'

    # 暗黙的な待機
    driver.implicitly_wait(5)
    driver.get(url)
    driver.maximize_window()
    return driver


def auto_login(driver):
    # ログインページ処理
    form = driver.find_element(by=By.CSS_SELECTOR, value='#signin_form')
    login_mail = form.find_element(by=By.NAME, value='email')
    login_passwd = form.find_element(by=By.NAME, value='password')

    # 認証情報
    with open('src/auth.json') as f:
        auth = json.load(f)

    my_mail = auth['my_mail']
    my_pass = auth['my_pass']

    # ログイン情報初期化
    login_mail.clear()
    login_passwd.clear()

    login_mail.send_keys(my_mail)
    login_passwd.send_keys(my_pass)

    # サインインボタンを押す
    sleep(0.5)
    btn = form.find_element(by=By.CSS_SELECTOR, value='#signin_btn')
    btn.click()
    sleep(2)

    # Slackアプリ起動確認メッセージのキャンセル
    pg.press('enter', presses=1, interval=0.1)
    sleep(1)

    # ブラウザでSlack起動
    pg.press('tab', presses=2, interval=0.1)
    pg.press('enter', presses=1, interval=0.1)

    # SlackのWebページが完全に立ち上がるまで8秒待機
    sleep(8)

In [77]:

driver = selenium_run()
auto_login(driver)

# メンバーディレクトリを表示
pg.hotkey('ctrl', 'shift', 'e')
sleep(5)

# 全員表示
pg.press('tab', presses=1, interval=0.1)
pg.press('enter', presses=1, interval=0.1)
pg.press('down', presses=2, interval=0.1)
pg.press('enter', presses=1, interval=0.1)
sleep(2)

# 50件/ページ として総ページ算出
pages = math.ceil(110 / 50)
print(f'計 {pages}ページ')

data = []
for page in range(pages):
    # メンバーのXPATH取得
    members_grid = driver.find_elements(by=By.CLASS_NAME, value='p-bp__grid_cell')
    print(len(members_grid))
    sleep(1)

    # 1人のメンバー取り出す
    for i, member_grid in enumerate(members_grid):
        member_grid.click()
        sleep(1)

        # 表示名の取得
        profile_name = driver.find_element(by=By.CLASS_NAME, value='p-r_member_profile__name__text').text

        # 連絡先取得
        email = driver.find_element(by=By.CLASS_NAME, value='c-link').get_attribute('href').split(':')[-1]

        # @XXXX名の取得
        buttons = driver.find_elements(by=By.CLASS_NAME, value='p-member_profile_buttons__button_body')
        buttons[-1].click()
        sleep(1)
        mention_name = driver.find_elements(by=By.CLASS_NAME, value='c-menu_item__label')[0].text.split(': ')[-1]
        driver.find_elements(by=By.CLASS_NAME, value='c-menu_item__label')[0].click()

        datum = {
            'profile_name': profile_name,
            'mention_name': mention_name,
            'email': email
        }

        data.append(datum)
        print(f'{page + 1}/{pages}ページ {i + 1} {profile_name}さんのデータ取得完了')
        sleep(0.5)
    print(f'{page + 1}/{pages}ページの取得完了しました')

    if page + 1 < pages:
        # 一番下まで移動
        pg.press('end', presses=3, interval=1)  # 大抵は1回で下へ遷移するが、たまに動かないため3回
        sleep(2)
        try:
            next_button = driver.find_element(by=By.XPATH, value='//*[@aria-label="次のページ"]')
            next_button.click()
            print(f'\n{page + 2}へのボタンをクリックしました')
        except Exception as e:
            print(e)
    else:
        pass

    sleep(2)

df_profile = pd.DataFrame(data)
print(df_profile)
print('完了')


計 3ページ
50
1/3ページ 1 Akito Nishimuraさんのデータ取得完了
1/3ページ 2 araさんのデータ取得完了
1/3ページ 3 Atsushiさんのデータ取得完了
1/3ページ 4 aya sugiyamaさんのデータ取得完了
1/3ページ 5 Daisukeさんのデータ取得完了
1/3ページ 6 eitaさんのデータ取得完了
1/3ページ 7 H.Yoshimuraさんのデータ取得完了
1/3ページ 8 hirota akitoさんのデータ取得完了
1/3ページ 9 imanyuさんのデータ取得完了
1/3ページ 10 jさんのデータ取得完了
1/3ページ 11 KANAUCHIさんのデータ取得完了
1/3ページ 12 Kaori Gendaさんのデータ取得完了
1/3ページ 13 Kayo Kubotaさんのデータ取得完了
1/3ページ 14 Keiko ISHIIさんのデータ取得完了
1/3ページ 15 Keisuke Tanaseさんのデータ取得完了
1/3ページ 16 ken1さんのデータ取得完了
1/3ページ 17 Kenichi Yoshidaさんのデータ取得完了
1/3ページ 18 kida kazushigeさんのデータ取得完了
1/3ページ 19 kiyomitsu OTAさんのデータ取得完了
1/3ページ 20 kohei ibukiさんのデータ取得完了
1/3ページ 21 mizuさんのデータ取得完了
1/3ページ 22 Nakagawaさんのデータ取得完了
1/3ページ 23 Nakayama Mさんのデータ取得完了
1/3ページ 24 Nitta Junさんのデータ取得完了
1/3ページ 25 Ryo Itabashiさんのデータ取得完了
1/3ページ 26 shigeru etouさんのデータ取得完了
1/3ページ 27 shingo.kanekoさんのデータ取得完了
1/3ページ 28 shinichi kofuneさんのデータ取得完了
1/3ページ 29 Shioda Kentoさんのデータ取得完了
1/3ページ 30 shiroさんのデータ取得完了
1/3ページ 31 shunさんのデータ取得完了
1/3ページ 32 Shunichi Morishimaさんのデータ取得完了
1/3ページ 33 Suba

In [78]:
df_profile



,profile_name,mention_name,email
0,Akito Nishimura,@nishimuraa,akito0884@gmail.com
1,ara,@ara,hagemashita.hagemasaremashita@gmail.com
2,Atsushi,@Atsushi,ahg.551001@gmail.com
3,aya sugiyama,@aya sugiyama,ayasugiyama2020@gmail.com
4,Daisuke,@Daisuke,mondo.piovoso@gmail.com
...,...,...,...
105,遠藤和宏,@遠藤和宏,endou_kzhr@jcom.zaq.ne.jp
106,野口雄気,@野口雄気,kuma10gzy@gmail.com
107,長崎 純一,@jun,jun117na@ymail.ne.jp
108,雨宮洋平,@雨宮洋平,kj2007ya@yahoo.co.jp


0

656